In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING LEBRARIES

In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import FalseNegatives
from tensorflow.keras.losses import BinaryCrossentropy

# LOADING DATA

In [ ]:
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.isnull().sum()

In [ ]:
palette = ["#c19277","#726a95"]
sns.set(palette=palette, font='sana-serif')
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams['font.family'] = 'sans-serif'

data.hist(figsize=(15,15), edgecolor='black')
plt.show()

In [ ]:
sns.countplot(data.DEATH_EVENT, palette=palette)
plt.show()

Maybe an oversampling method could be usefull, but i'm working with medical data, so syntetic data is not a good idea.

In [ ]:
plt.figure(figsize=(15,10))

mask = np.zeros_like(data.corr(), dtype= np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(data.corr(), annot=True, fmt='.3f', mask=mask, cmap='coolwarm')
plt.show()

The features age, ejection_fraction, serum_creatinine, serum_sodium, time are the most correlated with DEATH_EVENT.

Maybe just this features are enough for a good classification.

In [ ]:
principal_features = ['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium', 'time']

In [ ]:
plt.figure(figsize=(25,10))
sns.countplot(x='age', data=data)
plt.show()

I can see that most of the data can be represented by int, so lets turn this column to int.

In [ ]:
data['age'] = data.age.astype(np.int)

In [ ]:
plt.figure(figsize=(25,10))
sns.countplot(x='age', data=data)
plt.show()

In [ ]:
cmap = { 0 : "#c19277",1 : "#726a95"}
fig = px.histogram(data, x='age', color='DEATH_EVENT', marginal='violin', color_discrete_map= cmap, nbins=55)
fig.show()

In [ ]:
sns.boxplot(x='DEATH_EVENT', y='ejection_fraction', data=data, palette=palette)
plt.show()

In [ ]:
sns.boxplot(x='DEATH_EVENT', y='serum_creatinine', data=data, palette=palette)
plt.show()

In [ ]:
sns.boxplot(x='DEATH_EVENT', y='time', data=data, palette=palette)
plt.show()

# PREPROCESSING DATA

In [ ]:
feat_notbin = ['age','creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium','time']
feat_bin = ['anaemia',	'diabetes',	'high_blood_pressure',	'sex',	'smoking',	'DEATH_EVENT']

In [ ]:
data_nbin = data.drop(columns=feat_bin)
scaler = StandardScaler()
data_nbin = scaler.fit_transform(data_nbin)
data_nbin = pd.DataFrame(data_nbin, columns=feat_notbin)
data_nbin.head()

In [ ]:
plt.figure(figsize = (20,10))
sns.boxplot(data = data_nbin)
plt.xticks(rotation=90)
plt.show()

In [ ]:
data.drop(columns=feat_notbin, inplace=True)
data = data.join(data_nbin)
data.head()

# TRAINING WITH ALL DATA

### with all data

In [ ]:
X = data.drop(columns=['DEATH_EVENT'])
Y = data['DEATH_EVENT']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [ ]:
rnd_forest = RandomForestClassifier(n_estimators=600, n_jobs=-1)
rnd_forest.fit(X_train, Y_train)

Y_pred = rnd_forest.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - Random Forest')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
grad_boost = GradientBoostingClassifier(n_estimators=3, learning_rate= 0.5)
grad_boost.fit(X_train, Y_train)

Y_pred = grad_boost.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - GradientBoosting')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
Knn = KNeighborsClassifier(n_neighbors=6, n_jobs=-1)
Knn.fit(X_train, Y_train)

Y_pred = Knn.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - KNeighbors')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
model = Sequential()
model.add(Dense(units= 8, activation='relu', input_dim=12))
model.add(Dense(units= 10, activation='relu'))
model.add(Dense(units= 10, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units= 5, activation='relu'))
model.add(Dense(units= 5, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units= 2, activation='relu'))
model.add(Dense(units= 1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy', FalseNegatives()])

model.fit(X_train, Y_train, batch_size=32, epochs=500, validation_split=0.2)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.4)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - ANN')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

print(classification_report(Y_test,Y_pred))

# TRAINING WITH PRINCIPAL FEATURES

### with principal features

In [ ]:
X = data[principal_features]
Y = data["DEATH_EVENT"]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [ ]:
rnd_forest = RandomForestClassifier(n_estimators=600, n_jobs=-1)
rnd_forest.fit(X_train, Y_train)

Y_pred = rnd_forest.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - Random Forest')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
grad_boost = GradientBoostingClassifier(n_estimators=3, learning_rate= 0.5)
grad_boost.fit(X_train, Y_train)

Y_pred = grad_boost.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - GradientBoosting')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
Knn = KNeighborsClassifier(n_neighbors=6, n_jobs=-1)
Knn.fit(X_train, Y_train)

Y_pred = Knn.predict(X_test)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - KNeighbors')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
model = Sequential()
model.add(Dense(units= 8, activation='relu', input_dim=5))
model.add(Dense(units= 10, activation='relu'))
model.add(Dense(units= 10, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units= 5, activation='relu'))
model.add(Dense(units= 5, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units= 2, activation='relu'))
model.add(Dense(units= 1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy', FalseNegatives()])

model.fit(X_train, Y_train, batch_size=32, epochs=500, validation_split=0.2)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred = (Y_pred > 0.4)

fpr, tpr, thre = roc_curve(Y_test, Y_pred)

plt.figure(figsize=(10,5))

sns.lineplot(x=fpr, y=tpr)
plt.title('ROC curve - ANN')
plt.legend(['AUC : {:.4f}'.format(roc_auc_score(Y_test, Y_pred))])

plt.figure(figsize=(9,7))

cm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.show()

print(classification_report(Y_test,Y_pred))